# SciTeX Config Server - Configuration Management

This notebook demonstrates the Config Server's capabilities for managing SciTeX configuration files (PATH.yaml, PARAMS.yaml, COLORS.yaml) and extracting hardcoded values from code.

## 1. Setup and Connection

First, let's understand what the Config Server provides:

In [ ]:
# The Config Server provides tools for:
# 1. Extracting hardcoded values from code
# 2. Managing PATH.yaml, PARAMS.yaml, COLORS.yaml
# 3. Converting between config formats
# 4. Validating configuration files

# Example server connection (pseudo-code)
# from mcp import connect_to_server
# config_server = connect_to_server('scitex-config')

## 2. Extracting Hardcoded Values

The Config Server can analyze code and extract hardcoded values that should be in configuration files:

In [ ]:
# Example code with hardcoded values
messy_code = """
import pandas as pd
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('/home/user/data/experiment_2023.csv')
filtered = data[data['threshold'] > 0.75]

# Plot settings
plt.figure(figsize=(10, 6))
plt.plot(filtered['time'], filtered['signal'], color='#1f77b4', linewidth=2.5)
plt.xlabel('Time (s)', fontsize=14)
plt.ylabel('Signal', fontsize=14)
plt.title('Experiment Results', fontsize=16)
plt.savefig('/home/user/results/figure_1.png', dpi=300)

# Analysis parameters
window_size = 100
overlap = 0.5
sampling_rate = 1000
"""

print("Code with hardcoded values:")
print(messy_code)

In [ ]:
# Using Config Server to extract values
# result = config_server.extract_config(messy_code)

# Expected output structure:
extracted_config = {
    "paths": {
        "data_path": "/home/user/data/experiment_2023.csv",
        "output_path": "/home/user/results/figure_1.png",
    },
    "params": {
        "threshold": 0.75,
        "figure_size": [10, 6],
        "linewidth": 2.5,
        "font_sizes": {"labels": 14, "title": 16},
        "dpi": 300,
        "window_size": 100,
        "overlap": 0.5,
        "sampling_rate": 1000,
    },
    "colors": {"signal_color": "#1f77b4"},
}

print("Extracted configuration:")
for category, values in extracted_config.items():
    print(f"\n{category.upper()}:")
    for key, value in values.items():
        print(f"  {key}: {value}")

## 3. Generating Configuration Files

The server can generate proper SciTeX configuration files:

In [ ]:
# Generate PATH.yaml
path_yaml = """
# PATH.yaml - Project paths configuration
# Generated by SciTeX Config Server

data:
  experiment_data: ./data/experiment_2023.csv
  
output:
  figures: ./results/
  figure_1: ./results/figure_1.png
"""

print("Generated PATH.yaml:")
print(path_yaml)

In [ ]:
# Generate PARAMS.yaml
params_yaml = """
# PARAMS.yaml - Analysis parameters
# Generated by SciTeX Config Server

analysis:
  threshold: 0.75
  window_size: 100
  overlap: 0.5
  sampling_rate: 1000

visualization:
  figure_size: [10, 6]
  linewidth: 2.5
  dpi: 300
  font_sizes:
    labels: 14
    title: 16
"""

print("Generated PARAMS.yaml:")
print(params_yaml)

In [ ]:
# Generate COLORS.yaml
colors_yaml = """
# COLORS.yaml - Color scheme configuration
# Generated by SciTeX Config Server

signals:
  primary: '#1f77b4'
  secondary: '#ff7f0e'
  tertiary: '#2ca02c'

theme:
  background: '#ffffff'
  text: '#000000'
  grid: '#cccccc'
"""

print("Generated COLORS.yaml:")
print(colors_yaml)

## 4. Refactoring Code with Config

The server can refactor code to use configuration files:

In [ ]:
# Refactored code using config files
refactored_code = """
import scitex as stx

# Load configuration
CONFIG = stx.utils.load_config("./config/PATH.yaml")
PARAMS = stx.utils.load_config("./config/PARAMS.yaml")
COLORS = stx.utils.load_config("./config/COLORS.yaml")

def main(args):
    # Load data using config paths
    data = stx.io.load(CONFIG["data"]["experiment_data"])
    filtered = data[data['threshold'] > PARAMS["analysis"]["threshold"]]
    
    # Plot with config parameters
    fig, ax = stx.plt.subplots(figsize=PARAMS["visualization"]["figure_size"])
    ax.plot(filtered['time'], filtered['signal'], 
            color=COLORS["signals"]["primary"], 
            linewidth=PARAMS["visualization"]["linewidth"])
    
    # Set labels with config font sizes
    font_sizes = PARAMS["visualization"]["font_sizes"]
    ax.set_xyt('Time (s)', 'Signal', 'Experiment Results',
               xlabel_kwargs={'fontsize': font_sizes["labels"]},
               ylabel_kwargs={'fontsize': font_sizes["labels"]},
               title_kwargs={'fontsize': font_sizes["title"]})
    
    # Save with config path and dpi
    stx.io.save(fig, CONFIG["output"]["figure_1"], 
                dpi=PARAMS["visualization"]["dpi"])
    
    return 0
"""

print("Refactored code with configuration:")
print(refactored_code)

## 5. Advanced Config Management

The Config Server provides advanced features for managing configurations:

In [ ]:
# Config validation example
# result = config_server.validate_config('./config/PARAMS.yaml')

validation_result = {
    "valid": True,
    "warnings": [
        "Consider using relative paths in PATH.yaml",
        "Missing default values for some parameters",
    ],
    "suggestions": [
        "Add parameter ranges for validation",
        "Consider grouping related parameters",
    ],
}

print("Config validation result:")
print(f"Valid: {validation_result['valid']}")
print(f"\nWarnings:")
for warning in validation_result["warnings"]:
    print(f"  - {warning}")
print(f"\nSuggestions:")
for suggestion in validation_result["suggestions"]:
    print(f"  - {suggestion}")

In [ ]:
# Config merging example
base_params = {
    "analysis": {"threshold": 0.5, "window_size": 50},
    "visualization": {"dpi": 150},
}

experiment_params = {
    "analysis": {
        "threshold": 0.75,  # Override
        "sampling_rate": 1000,  # New parameter
    },
    "visualization": {
        "figure_size": [10, 6]  # New parameter
    },
}

# Merge configurations
# merged = config_server.merge_configs(base_params, experiment_params)

merged_params = {
    "analysis": {
        "threshold": 0.75,  # Overridden
        "window_size": 50,  # From base
        "sampling_rate": 1000,  # From experiment
    },
    "visualization": {
        "dpi": 150,  # From base
        "figure_size": [10, 6],  # From experiment
    },
}

print("Merged configuration:")
import json

print(json.dumps(merged_params, indent=2))

## 6. Environment-Specific Configurations

Managing different configurations for different environments:

In [ ]:
# Environment-specific config structure
env_configs = {
    "development": {
        "paths": {"data": "./test_data/", "output": "./test_output/"},
        "params": {
            "debug": True,
            "sampling_rate": 100,  # Lower for testing
            "max_iterations": 10,
        },
    },
    "production": {
        "paths": {"data": "/data/experiments/", "output": "/results/production/"},
        "params": {"debug": False, "sampling_rate": 1000, "max_iterations": 1000},
    },
    "hpc": {
        "paths": {"data": "$SCRATCH/data/", "output": "$SCRATCH/results/"},
        "params": {
            "debug": False,
            "sampling_rate": 10000,
            "max_iterations": 10000,
            "parallel": True,
            "n_cores": 32,
        },
    },
}

print("Environment configurations:")
for env, config in env_configs.items():
    print(f"\n{env.upper()}:")
    print(json.dumps(config, indent=2))

## 7. Config Templates and Scaffolding

The Config Server can generate templates for new projects:

In [ ]:
# Generate project config scaffold
# scaffold = config_server.generate_scaffold('neuroscience_project')

scaffold_structure = {
    "config/": {
        "PATH.yaml": """# Project paths
data:
  raw: ./data/raw/
  processed: ./data/processed/
  
output:
  figures: ./figures/
  results: ./results/
  logs: ./logs/
""",
        "PARAMS.yaml": """# Analysis parameters
preprocessing:
  sampling_rate: 1000
  filter:
    low_freq: 0.1
    high_freq: 100
    
analysis:
  window_size: 1000
  overlap: 0.5
  
visualization:
  figure_size: [10, 6]
  dpi: 300
""",
        "COLORS.yaml": """# Color scheme
channels:
  ch1: '#1f77b4'
  ch2: '#ff7f0e'
  ch3: '#2ca02c'
  ch4: '#d62728'
  
conditions:
  control: '#666666'
  treatment: '#e74c3c'
""",
    }
}

print("Generated config scaffold:")
for path, content in scaffold_structure["config/"].items():
    print(f"\n=== {path} ===")
    print(content)

## 8. Dynamic Config Loading

Advanced patterns for dynamic configuration:

In [ ]:
# Dynamic config loader with inheritance
dynamic_config_code = '''
import scitex as stx
import os

class ConfigManager:
    """Dynamic configuration management with SciTeX"""
    
    def __init__(self, config_dir="./config", env=None):
        self.config_dir = config_dir
        self.env = env or os.getenv('SCITEX_ENV', 'development')
        self._cache = {}
        
    def load(self, config_type):
        """Load configuration with environment overrides"""
        if config_type in self._cache:
            return self._cache[config_type]
            
        # Load base config
        base_path = f"{self.config_dir}/{config_type}.yaml"
        config = stx.utils.load_config(base_path)
        
        # Apply environment-specific overrides
        env_path = f"{self.config_dir}/{config_type}.{self.env}.yaml"
        if os.path.exists(env_path):
            env_config = stx.utils.load_config(env_path)
            config = stx.utils.merge_dicts(config, env_config)
            
        self._cache[config_type] = config
        return config
        
    def get_path(self, key):
        """Get path with automatic expansion"""
        paths = self.load('PATH')
        path = stx.utils.get_nested(paths, key)
        return os.path.expandvars(path)
        
    def get_param(self, key, default=None):
        """Get parameter with default fallback"""
        params = self.load('PARAMS')
        return stx.utils.get_nested(params, key, default)

# Usage
config = ConfigManager(env='production')
data_path = config.get_path('data.raw')
threshold = config.get_param('analysis.threshold', 0.5)
'''

print("Dynamic config manager example:")
print(dynamic_config_code)

## Summary

The SciTeX Config Server provides powerful tools for:

1. **Extraction**: Automatically extract hardcoded values from existing code
2. **Generation**: Create proper YAML configuration files
3. **Refactoring**: Transform code to use configuration files
4. **Validation**: Ensure configurations are valid and complete
5. **Environment Management**: Handle different configs for different environments
6. **Scaffolding**: Generate config templates for new projects
7. **Dynamic Loading**: Advanced patterns for flexible configuration

This enables better reproducibility, easier parameter tuning, and cleaner code organization in scientific computing projects.